In [1]:
import pandas as pd


In [2]:
df = pd.read_csv('../data/clean_game_data.csv')

print(df.head())

   Column       Side  Region  Year  Game ID          Team  Victory  \
0       0  Blue Side  WORLDS  2023    53622  Weibo Gaming    False   
1       1  Blue Side  WORLDS  2023    53622  Weibo Gaming    False   
2       2  Blue Side  WORLDS  2023    53622  Weibo Gaming    False   
3       3  Blue Side  WORLDS  2023    53622  Weibo Gaming    False   
4       4  Blue Side  WORLDS  2023    53622  Weibo Gaming    False   

   Game Duration  Player     Role  ...  Quadra kills  Penta kills  GD@15  \
0         30.517  TheShy      TOP  ...             0            0   -243   
1         30.517  WeiWei   JUNGLE  ...             0            0    212   
2         30.517  xiaohu      MID  ...             0            0    363   
3         30.517   Crisp      ADC  ...             0            0  -1002   
4         30.517   Light  SUPPORT  ...             0            0    435   

   CSD@15  XPD@15  LVLD@15  Damage dealt to turrets  Total heal  \
0     -30     -64        0                      968    

In [7]:
# DF has the stats for each game for each player of each team
df = pd.read_csv('../data/clean_game_data.csv')

# Context : League of Legends Pro play
# Creating team elo ratings + storing elo evolution (per year)

# Initial elo ratings

teams = df['Team'].unique()

teams = [team for team in teams if team not in ['DAMWON Gaming', 'DWG KIA']] # DAMWON Gaming and DWG KIA are the same team, but the name changed in 2021
elo_ratings = {team: 1000 for team in teams}


# Storing elo evolution

elo_evolution = {team: [] for team in teams}

# Updating elo ratings, going in chronological order (Game ID increases with time)
# Reordering the dataframe to have the games in chronological order

df = df.sort_values(by='Game ID')
df = df.groupby('Game ID')



for game_id, game in df:
    for team in teams:
        elo_evolution[team].append(elo_ratings[team])

    # Updating elo ratings
    blue_line = game[(game['Role'] == 'TOP') & (game['Side'] == 'Blue Side')]
    blue_team = blue_line['Team'].values[0]
    blue_victory = blue_line['Victory'].values[0]

    red_line = game[(game['Role'] == 'TOP') & (game['Side'] == 'Red Side')]
    red_team = red_line['Team'].values[0]

    if blue_victory:
        winner = blue_team
        loser = red_team
    else:
        winner = red_team
        loser = blue_team

    # Dplus KIA (latest name, sounds like DisneyPlus) and DWG KIA and Damwon Gaming are the same team, but the name changed in 2021
    # Special rule
    if winner == 'DAMWON Gaming':
        winner = 'Dplus KIA'
    if loser == 'DAMWON Gaming':
        loser = 'Dplus KIA'
    if winner == 'DWG KIA':
        winner = 'Dplus KIA'
    if loser == 'DWG KIA':
        loser = 'Dplus KIA'
    
    # Elo update formula

    elo_winner = elo_ratings[winner]
    elo_loser = elo_ratings[loser]

    elo_ratings[winner] = elo_winner + 32 * (1 - 1 / (1 + 10 ** ((elo_loser - elo_winner) / 400)))
    elo_ratings[loser] = elo_loser + 32 * (0 - 1 / (1 + 10 ** ((elo_winner - elo_loser) / 400)))

# Storing the elo evolution in a dataframe

elo_df = pd.DataFrame(elo_evolution)
elo_df.to_csv('../data/elo_evolution.csv', index=False)

print(elo_df.head())

   Weibo Gaming      T1  JD Gaming  Bilibili Gaming  LNG Esports  KT Rolster  \
0        1000.0  1000.0     1000.0           1000.0       1000.0      1000.0   
1        1000.0  1000.0     1000.0           1000.0       1000.0      1000.0   
2        1000.0  1000.0     1000.0           1000.0       1000.0      1000.0   
3        1000.0  1000.0     1000.0           1000.0       1000.0      1000.0   
4        1000.0  1000.0     1000.0           1000.0       1000.0      1000.0   

   Gen.G eSports     NRG  G2 Esports  Fnatic  ...  Papara SuperMassive  \
0         1000.0  1000.0      1000.0  1000.0  ...               1000.0   
1         1000.0  1000.0      1000.0  1000.0  ...               1000.0   
2         1000.0  1000.0      1000.0  1000.0  ...               1000.0   
3         1000.0  1000.0      1000.0  1000.0  ...               1000.0   
4         1000.0  1000.0      1000.0  1000.0  ...               1000.0   

   Legacy eSports  Rainbow7  INTZ e-Sports  V3 Esports  Team Dynamics  \
0

In [8]:
# Dplus KIA (latest name, sounds like DisneyPlus) and DWG KIA and Damwon Gaming are the same team, but the name changed in 2021

elo_df = pd.read_csv('../data/elo_evolution.csv')



In [9]:
# Plotting top 10 team (latest) elo evolution curve

import plotly.express as px

elo_df['Game ID'] = range(len(elo_df))
elo_df = elo_df.set_index('Game ID')

top_elo = elo_df.iloc[-1].sort_values(ascending=False).head(10)

top_elo_df = elo_df[top_elo.index]

fig = px.line(top_elo_df, title='Top 10 team elo evolution', labels={'value': 'Elo rating', 'index': 'Game ID'})
fig.show()
